# **Práctica 2: Clasificación Multi-Intancia**
* **Master:** Inteligencia Computacional y Internet de las Cosas
* **Asignatura:** Clasificación Non-Convencional
* **Estudiante:** Mabrouka Salmi
* **Fecha:** 14-03-2023
* **El Cuaderno Colab de la [Práctica](https://colab.research.google.com/drive/16LAwhl8i_GqlfQkX-4AEdxL0QN12KiRj?usp=sharing)**



---


**Optamos por la segunda opción: comparación de métodos para esta práctica.**

**Enfoques/Métodos comparados**

✔ Enfoque Instance Space
▶ `APR`

✔ Enfoque de espacio embebido

▶ `Discriminative Mapping Using SVC`

▶ `MILES`

**Conjunto de datos utilizado:** ◾ CorelAfrican Dataset ◾ CorelDesserts Dataset ◾ CorelCars Dataset. Descargado de  [multipleinstance](http://www.multipleinstancelearning.com/datasets/)

**Bibliotecas utilizadas:** Multi-Instance Learning Library para Python [mil](https://github.com/rosasalberto/mil/tree/master)


---



> 🔰 **Nota:**  Intenté traducir los textos del cuaderno, sin embargo no pude comprobar si mantiene el mismo significado, agradeceré su comprensión y tiempo  

## **1. Packages for Mulit-Instance Learning**

In [ ]:
# Requisitos para MISVM: Máquinas de vectores soporte de múltiples instancias
import numpy as np
import scipy
import cvxopt

[misvm](https://github.com/garydoranjr/misvm)

In [ ]:
# en caso necesario:
# pip install numpy
# pip install scipy
# pip install cvxopt
%pip install -e git+https://github.com/garydoranjr/misvm.git#egg=misvm

Obtaining misvm from git+https://github.com/garydoranjr/misvm.git#egg=misvm
  Cloning https://github.com/garydoranjr/misvm.git to ./src/misvm
  Running command git clone --filter=blob:none --quiet https://github.com/garydoranjr/misvm.git /content/src/misvm
  Resolved https://github.com/garydoranjr/misvm.git to commit b2118fe04d98c00436bdf8a0e4bbfb6082c5751c
  Preparing metadata (setup.py) ... done
  Running setup.py develop for misvm


In [ ]:
# Requisito para:  mil: multiple instance learning library for Python
import  sklearn
import tensorflow

Multiple instance learning library:
[mil_github](https://github.com/rosasalberto/mil)

In [ ]:
# install mil
%pip install mil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 24.2 MB/s eta 0:00:00


## **2. Selected Datasets**




 Corel son problemas de clasificación de escenas. Cada bolsa es una imagen, cada instancia
es un fragmento de esa imagen. Las imágenes representan escenas de postres, coches, etc. Originalmente, tiene 20 clases, se generan 20 conjuntos de datos
utilizando el enfoque de uno contra todos, utilizamos tres de ellos:

* **Corel_African**
* **Corel_Desserts**
* **Corel_Cars**

### **Carga del conjunto de datos CorelAfrican**

In [ ]:
import pandas as pd

In [ ]:
# https://www.dropbox.com/sh/agpyhqsxcrkxx32/AABO9RC8LZzE--XQskv2Rb3ua/Web1.csv?dl=0
n=10
column_names = ['bagClass', 'bagID'] + [f'f{i}' for i in range(2, n+1)]
CorelAfrican = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CorelAfrican.csv",header=None,names=column_names)
CorelAfrican.head()

,bagClass,bagID,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,1,1,4.821059,0.593897,1.361879,2.706519,3.575774,1.324719,0.375802,0.283057,0.181409
1,1,1,7.199910,1.423148,3.127087,3.913973,3.997469,2.024607,0.157150,0.091908,0.066273
2,1,1,6.818762,-0.670180,-0.492980,2.938068,2.915321,1.422168,0.188361,0.118776,0.080730
3,1,1,5.283375,10.266975,-0.115075,2.131377,1.692391,0.808610,-0.017256,-0.004026,0.026912
4,1,1,5.446272,2.814494,1.619185,3.176151,3.389741,1.416670,0.250244,0.178293,0.122883


In [ ]:
import numpy as np
import pandas as pd

# Group instances by bag_id
bags_grouped = CorelAfrican.groupby('bagID')

# Initialize lists to store bags and labels
bagsAfrican = []
labelsAfrican = []

# Iterate over each bag
for bag_id, bag_data in bags_grouped:
    # Extract bag features (excluding 'bag_id' and 'bag_class')
    bag_features = bag_data.iloc[:, 2:].to_numpy()

    # Append bag features to the bags list
    bagsAfrican.append(bag_features)

    # Extract and append label for the bag
    bag_label = bag_data['bagClass'].iloc[0]
    labelsAfrican.append(bag_label)
print("shape of bags",len(bagsAfrican))
print(bagsAfrican[0:2])


shape of bags 2000
[array([[ 4.82105904e+00,  5.93896770e-01,  1.36187897e+00,
         2.70651865e+00,  3.57577419e+00,  1.32471943e+00,
         3.75801712e-01,  2.83056557e-01,  1.81408733e-01],
       [ 7.19990997e+00,  1.42314796e+00,  3.12708664e+00,
         3.91397309e+00,  3.99746919e+00,  2.02460718e+00,
         1.57149568e-01,  9.19084700e-02,  6.62729220e-02],
       [ 6.81876221e+00, -6.70179653e-01, -4.92979574e-01,
         2.93806839e+00,  2.91532135e+00,  1.42216802e+00,
         1.88361302e-01,  1.18776314e-01,  8.07300430e-02],
       [ 5.28337517e+00,  1.02669746e+01, -1.15075004e-01,
         2.13137674e+00,  1.69239104e+00,  8.08609784e-01,
        -1.72561110e-02, -4.02617500e-03,  2.69116000e-02],
       [ 5.44627152e+00,  2.81449413e+00,  1.61918545e+00,
         3.17615056e+00,  3.38974070e+00,  1.41667044e+00,
         2.50244081e-01,  1.78293377e-01,  1.22883223e-01],
       [ 7.86114578e+00,  1.20846357e+00,  1.26116142e+00,
         2.40052319e+00,  2.897



###  **Carga del conjunto de datos CorelDessert**


In [ ]:
# https://www.dropbox.com/sh/agpyhqsxcrkxx32/AADKf5oD7L0tcKIjJVJRvCdCa/CorelDesserts.csv?e=1&dl=0
n=10
column_names = ['bagClass', 'bagID'] + [f'f{i}' for i in range(2, n+1)]
CorelDesserts = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CorelDesserts.csv",header=None,names=column_names)

In [ ]:
import numpy as np
import pandas as pd

# Group instances by bag_id
bags_grouped = CorelDesserts.groupby('bagID')

# Initialize lists to store bags and labels
bagsDesserts = []
labelsDesserts = []

# Iterate over each bag
for bag_id, bag_data in bags_grouped:
    # Extract bag features (excluding 'bag_id' and 'bag_class')
    bag_features = bag_data.iloc[:, 2:].to_numpy()

    # Append bag features to the bags list
    bagsDesserts.append(bag_features)

    # Extract and append label for the bag
    bag_label = bag_data['bagClass'].iloc[0]
    labelsDesserts.append(bag_label)
print("shape of bags",len(bagsDesserts))
print("ten first elements of the bags= ",bagsDesserts[0:3])


shape of bags 2000
ten first elements of the bags=  [array([[ 8.11135330e+00, -3.54739165e-01, -8.51226997e-01,
         1.99917865e+00,  1.36801612e+00,  9.03144479e-01,
        -1.70224980e-02, -1.15696000e-04,  2.90671980e-02],
       [ 5.19403648e+00, -5.25066948e-01, -3.14554939e+00,
         7.05675030e+00,  3.73167300e+00,  3.08188486e+00,
         2.41932290e-02,  3.33829340e-02,  4.45216750e-02]]), array([[ 3.60204239, -0.23663404, -4.23494911,  6.79396391,  4.9840517 ,
         3.83226657,  0.14373371,  0.13427412,  0.10137582],
       [ 8.85350342,  0.09761895, -0.97469349,  1.55583453,  1.34251416,
         0.83084971,  0.01824229,  0.02006292,  0.03690241],
       [ 6.57803497,  0.11510569, -1.60641136,  7.65902615,  4.22317457,
         3.20336366,  0.03843562,  0.037366  ,  0.04528224]]), array([[ 3.12990093e+00, -2.40804458e-01, -1.62116337e+00,
         1.09429893e+01,  8.81615829e+00,  6.60587835e+00,
         1.61717236e-01,  1.59350216e-01,  1.25094160e-01],
       

### **Carga del conjunto de datos CorelDessert CorelCars**

In [ ]:
# The below link is used to download the CorelCars dataset
# https://www.dropbox.com/sh/agpyhqsxcrkxx32/AACUJTcubiotocJBs18_WCWca/CorelCars.csv?e=1&dl=0
n=10
column_names = ['bagClass', 'bagID'] + [f'f{i}' for i in range(2, n+1)]
CorelCars = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CorelCars.csv",header=None,names=column_names)

In [ ]:
import numpy as np
import pandas as pd

# Group instances by bag_id
bags_grouped = CorelCars.groupby('bagID')

# Initialize lists to store bags and labels
bagsCars = []
labelsCars = []

# Iterate over each bag
for bag_id, bag_data in bags_grouped:
    # Extract bag features (excluding 'bag_id' and 'bag_class')
    bag_features = bag_data.iloc[:, 2:].to_numpy()

    # Append bag features to the bags list
    bagsCars.append(bag_features)

    # Extract and append label for the bag
    bag_label = bag_data['bagClass'].iloc[0]
    labelsCars.append(bag_label)
print("shape of bags",len(bagsCars))
print("ten first elements of the bags= ",bagsCars[0:3])

shape of bags 2000
ten first elements of the bags=  [array([[ 6.50740814, -1.57546387, -1.74076691,  0.67659706,  0.54493767,
         0.38886175,  0.05265258,  0.05678634,  0.0572464 ],
       [ 2.97259235,  0.09049149,  1.01784487,  8.9087944 , 10.07102203,
         5.95166969,  0.24136852,  0.16596581,  0.11356009],
       [ 6.62781601,  0.65450044,  1.65608788,  8.00679207,  7.61528635,
         4.18144512,  0.01987228,  0.01430246,  0.03343308]]), array([[ 2.52320061,  0.03168821,  0.26638176,  6.94873238,  6.84407043,
         3.60523128,  0.06902869,  0.05092875,  0.0506701 ],
       [ 8.3212677 , -0.81319056, -0.36880827,  1.6646502 ,  1.61368895,
         0.9684096 ,  0.16354714,  0.10412321,  0.07298077],
       [ 5.62301636,  0.11746101,  0.5050333 ,  8.95795441,  8.78778935,
         4.35319853,  0.07932695,  0.04199187,  0.04344482]]), array([[ 4.71677818, -0.07544677,  1.1325223 ,  9.88647461, 11.00667286,
         7.14415264,  0.0959656 ,  0.07455373,  0.06725168],
     

Instalación de la biblioteca Multiple-Instance Support Vector Machines: MISVM

In [ ]:
%pip uninstall misvm

In [ ]:
!pip install git+https://github.com/garydoranjr/misvm.git#egg=misvm

  Cloning https://github.com/garydoranjr/misvm.git to /tmp/pip-install-jsqiu1sk/misvm_7a42026b0a1640e4832d9805ee069c99
  Running command git clone --filter=blob:none --quiet https://github.com/garydoranjr/misvm.git /tmp/pip-install-jsqiu1sk/misvm_7a42026b0a1640e4832d9805ee069c99
  Resolved https://github.com/garydoranjr/misvm.git to commit b2118fe04d98c00436bdf8a0e4bbfb6082c5751c
  Preparing metadata (setup.py) ... done
  Created wheel for misvm: filename=misvm-1.0-py3-none-any.whl size=27219 sha256=0a631f00f8210aeddf73f331818f272309f05f288988edc5ca3cd44cbf64ee1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-qrg1t64o/wheels/ff/d7/4e/40bcb8956acbfbd7f287b238ca0155b468c074571e5eca26b4
Successfully built misvm


### **Bibliotecas relacionadas**

In [ ]:
from mil.bag_representation import MILESMapping, DiscriminativeMapping, ArithmeticMeanBagRepresentation, \
                                   MedianBagRepresentation, GeometricMeanBagRepresentation, MinBagRepresentation, \
                                   MaxBagRepresentation, MeanMinMaxBagRepresentation

In [ ]:
from mil.bag_representation import MILESMapping

In [ ]:
import numpy as np
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
!pip install keras==2.8   # Necesario para el correcto funcionamiento de los métodos de la biblioteca mil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.8.0 which is incompatible.


In [ ]:
!pip install tensorflow==2.8  # Necesario para el correcto funcionamiento de los métodos de la biblioteca mil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.

In [ ]:
# importar estrategia de validación
from mil.validators import LeaveOneOut, KFold
# importando el modelo final, que en este caso es el clasificador SVC de sklearn
from mil.models import SVC
# importando trainer
from mil.trainer import Trainer
# importando preprocessing
from mil.preprocessing import StandarizerBagsList
# importando métricas, que en este caso son de tf keras metrics
from mil.metrics import AUC, BinaryAccuracy, Sensibility, Specificity

In [ ]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.9 MB/s eta 0:00:00


## **3.1.  APR: Instance-based Classifier**


El algoritmo APR se basa en la hipótesis SMI, que establece que cada bolsa positiva contiene al menos una instancia positiva, mientras que en cada bolsa negativa todas las instancias son negativas.  Este algoritmo utiliza Axis-Parallel Rectangle (APR) para estimar un clasificador a nivel de instancia.

In [ ]:
from mil.models import APR


### **CorelAfrican Dataset**

In [ ]:

from sklearn.model_selection import train_test_split

bag_indices = range(len(bagsAfrican))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsAfrican, labelsAfrican, bag_indices, test_size=0.2, random_state=42
)

In [ ]:
trainer =Trainer()
valid =valid = KFold(n_splits=5, shuffle = True)
metrics = [AUC, BinaryAccuracy, Sensibility, Specificity]
model = APR()
pipeline = [('scale', StandarizerBagsList())]

trainer.prepare(model, preprocess_pipeline=pipeline, metrics=metrics)

In [ ]:
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

Iteration: 1, APR size: 13.74, Discriminative features: 9
---Grow and discriminate has converged---

1/5 [=====>........................] - ETA: 5s - train_auc: 0.5338 - train_binaryaccuracy: 0.1156 - train_sensibility: 1.0000 - train_specificity: 0.0675 - val_auc: 0.5230 - val_binaryaccuracy: 0.0938 - val_sensibility: 1.0000 - val_specificity: 0.0461Iteration: 1, APR size: 14.03, Discriminative features: 9
---Grow and discriminate has converged---

2/5 [===========>..................] - ETA: 2s - train_auc: 0.5338 - train_binaryaccuracy: 0.1164 - train_sensibility: 1.0000 - train_specificity: 0.0676 - val_auc: 0.5443 - val_binaryaccuracy: 0.1312 - val_sensibility: 1.0000 - val_specificity: 0.0885Iteration: 1, APR size: 13.88, Discriminative features: 9
---Grow and discriminate has converged---

3/5 [=================>............] - ETA: 1s - train_auc: 0.5346 - train_binaryaccuracy: 0.1180 - train_sensibility: 1.0000 - train_specificity: 0.0692 - val_auc: 0.5344 - val_binaryaccuracy:

In [ ]:
print("The mean of the binaryaccuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binaryaccuracy =  0.11749999


[{'auc': 0.52302635,
  'binaryaccuracy': 0.09375,
  'sensibility': 1.0,
  'specificity': 0.04605263},
 {'auc': 0.5442623,
  'binaryaccuracy': 0.13125,
  'sensibility': 1.0,
  'specificity': 0.08852459},
 {'auc': 0.5344262,
  'binaryaccuracy': 0.1125,
  'sensibility': 1.0,
  'specificity': 0.06885246},
 {'auc': 0.52134097,
  'binaryaccuracy': 0.146875,
  'sensibility': 0.95238096,
  'specificity': 0.09030101},
 {'auc': 0.52950823,
  'binaryaccuracy': 0.103125,
  'sensibility': 1.0,
  'specificity': 0.05901639}]

In [ ]:
labels_predicted = trainer.predict(bags_test)
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.5353403,
 'binaryaccuracy': 0.1125,
 'sensibility': 1.0,
 'specificity': 0.070680626}

**Interpretation:** The APR follows the Statndard MI assumption , which states every positive bag contains at least on positive instance. As such a bag contining at least one positive instance is classified as positive bag. The asumption is leading to misclassifying the majority of the negative bags where only 7% of the negative bags are correctly classified. While 100% of the positive bags are correctly classified. As a results the high misclassification of negative bags yielding to a very low value of accuracy associated with a low area under the ROC curve indicating that the discrimination between the classes of ARP classifier is almost equal as a random discrimination.

### **CorelDesserts**

In [ ]:

from sklearn.model_selection import train_test_split

bag_indices = range(len(bagsDesserts))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsDesserts, labelsDesserts, bag_indices, test_size=0.2, random_state=42
)

In [ ]:
trainer =Trainer()
valid =valid = KFold(n_splits=5, shuffle = True)
metrics = [AUC, BinaryAccuracy, Sensibility, Specificity]
model = APR()
pipeline = [('scale', StandarizerBagsList())]

trainer.prepare(model, preprocess_pipeline=pipeline, metrics=metrics)

In [ ]:
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

Iteration: 1, APR size: 12.54, Discriminative features: 9
---Grow and discriminate has converged---

1/5 [=====>........................] - ETA: 2s - train_auc: 0.6462 - train_binaryaccuracy: 0.3289 - train_sensibility: 1.0000 - train_specificity: 0.2924 - val_auc: 0.6612 - val_binaryaccuracy: 0.3562 - val_sensibility: 1.0000 - val_specificity: 0.3224Iteration: 1, APR size: 12.56, Discriminative features: 9
---Grow and discriminate has converged---

2/5 [===========>..................] - ETA: 1s - train_auc: 0.6469 - train_binaryaccuracy: 0.3313 - train_sensibility: 1.0000 - train_specificity: 0.2937 - val_auc: 0.6503 - val_binaryaccuracy: 0.3313 - val_sensibility: 1.0000 - val_specificity: 0.3007Iteration: 1, APR size: 14.09, Discriminative features: 9
---Grow and discriminate has converged---

3/5 [=================>............] - ETA: 1s - train_auc: 0.5942 - train_binaryaccuracy: 0.2297 - train_sensibility: 1.0000 - train_specificity: 0.1885 - val_auc: 0.5957 - val_binaryaccuracy:

In [ ]:
print("The mean of the binaryaccuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binaryaccuracy =  0.3125


[{'auc': 0.6611842,
  'binaryaccuracy': 0.35625,
  'sensibility': 1.0,
  'specificity': 0.3223684},
 {'auc': 0.6503268,
  'binaryaccuracy': 0.33125,
  'sensibility': 1.0,
  'specificity': 0.3006536},
 {'auc': 0.59570956,
  'binaryaccuracy': 0.234375,
  'sensibility': 1.0,
  'specificity': 0.19141914},
 {'auc': 0.6063281,
  'binaryaccuracy': 0.30625,
  'sensibility': 0.9444444,
  'specificity': 0.26821193},
 {'auc': 0.6207532,
  'binaryaccuracy': 0.334375,
  'sensibility': 0.9411765,
  'specificity': 0.30033004}]

In [ ]:
labels_predicted = trainer.predict(bags_test)
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.607039,
 'binaryaccuracy': 0.3,
 'sensibility': 0.9444444,
 'specificity': 0.2696335}

**Interpretation:** The ARP classifier is highly biased towards the positive bags due to the asumption is based on, this leading to ignoring the negative class bags and more attention is allocated to the positive class bags. Consequently, the accuracy of the classifier on the test data is insignificant which is expected due to the misclassification of the negative bags as positive bags. Besides, the area under the ROC curve of the ARP classifier is average, considerng that the algorithm is correctly classifying the majority of the positive bags.

### **CorelCars**

In [ ]:

from sklearn.model_selection import train_test_split

bag_indices = range(len(bagsCars))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsCars, labelsCars, bag_indices, test_size=0.2, random_state=42
)

In [ ]:
trainer =Trainer()
valid =valid = KFold(n_splits=5, shuffle = True)
metrics = [AUC, BinaryAccuracy, Sensibility, Specificity]
model = APR()
pipeline = [('scale', StandarizerBagsList())]

trainer.prepare(model, preprocess_pipeline=pipeline, metrics=metrics)

In [ ]:
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

Iteration: 1, APR size: 14.6, Discriminative features: 9
---Grow and discriminate has converged---

1/5 [=====>........................] - ETA: 1s - train_auc: 0.5641 - train_binaryaccuracy: 0.1711 - train_sensibility: 1.0000 - train_specificity: 0.1282 - val_auc: 0.5385 - val_binaryaccuracy: 0.1781 - val_sensibility: 0.9474 - val_specificity: 0.1296Iteration: 1, APR size: 15.15, Discriminative features: 9
---Grow and discriminate has converged---

2/5 [===========>..................] - ETA: 1s - train_auc: 0.5409 - train_binaryaccuracy: 0.1312 - train_sensibility: 1.0000 - train_specificity: 0.0818 - val_auc: 0.5358 - val_binaryaccuracy: 0.1094 - val_sensibility: 1.0000 - val_specificity: 0.0717Iteration: 1, APR size: 15.22, Discriminative features: 9
---Grow and discriminate has converged---

3/5 [=================>............] - ETA: 1s - train_auc: 0.5380 - train_binaryaccuracy: 0.1250 - train_sensibility: 1.0000 - train_specificity: 0.0759 - val_auc: 0.5359 - val_binaryaccuracy: 

In [ ]:
print("The mean of the binaryaccuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binaryaccuracy =  0.16437499


[{'auc': 0.53846824,
  'binaryaccuracy': 0.178125,
  'sensibility': 0.94736844,
  'specificity': 0.1295681},
 {'auc': 0.5358306,
  'binaryaccuracy': 0.109375,
  'sensibility': 1.0,
  'specificity': 0.07166124},
 {'auc': 0.5359477,
  'binaryaccuracy': 0.1125,
  'sensibility': 1.0,
  'specificity': 0.07189543},
 {'auc': 0.6026014,
  'binaryaccuracy': 0.3,
  'sensibility': 0.9411765,
  'specificity': 0.2640264},
 {'auc': 0.53322256,
  'binaryaccuracy': 0.121875,
  'sensibility': 1.0,
  'specificity': 0.06644518}]

In [ ]:
labels_predicted = trainer.predict(bags_test)
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.53795815,
 'binaryaccuracy': 0.1175,
 'sensibility': 1.0,
 'specificity': 0.07591623}

**Interpretation:** The instance sapce based method APR is poorly performing on the `CorelCars` dataset pretty similar to it performance on the `CarsAfrican` dataset. Although the APR model is correctly classifying all the positve bags in the test data is only identified 11% of the negative class. That is resulting in very low values of  `binaryaccuracy` and `area under the ROC curve` indicating the low classification performance of the classifer.

## **3.2.  Discriminative Mapping Using SVC: Embedded-Space Method**

### **CorelAfrican Dataset**

In [ ]:

from sklearn.model_selection import train_test_split

bag_indices = range(len(bagsAfrican))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsAfrican, labelsAfrican, bag_indices, test_size=0.2, random_state=42
)


In [ ]:
# instanciar entrenador
trainer = Trainer()

# preparar trainer
metrics = [AUC, BinaryAccuracy,Sensibility, Specificity]
model = SVC(kernel='linear', C=1, class_weight='balanced')
pipeline = [('scale', StandarizerBagsList()), ('disc_mapping', MILESMapping())]
trainer.prepare(model, preprocess_pipeline=pipeline ,metrics=metrics)

In [ ]:
# fitting trainer
valid = KFold(n_splits=5, shuffle = True)
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

5/5 [==============================] - 36s 7s/step - train_auc: 0.9749 - train_binaryaccuracy: 0.9523 - train_sensibility: 1.0000 - train_specificity: 0.9498 - val_auc: 0.7496 - val_binaryaccuracy: 0.8938 - val_sensibility: 0.5882 - val_specificity: 0.9109


In [ ]:
# imprimir los resultados de la validación de cada pliegue
print("La media de la binary accuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binary accuracy =  0.9150001


[{'auc': 0.7939891,
  'binaryaccuracy': 0.909375,
  'sensibility': 0.6666667,
  'specificity': 0.9213115},
 {'auc': 0.75166667,
  'binaryaccuracy': 0.928125,
  'sensibility': 0.55,
  'specificity': 0.9533333},
 {'auc': 0.9021858,
  'binaryaccuracy': 0.934375,
  'sensibility': 0.8666667,
  'specificity': 0.9377049},
 {'auc': 0.7939891,
  'binaryaccuracy': 0.909375,
  'sensibility': 0.6666667,
  'specificity': 0.9213115},
 {'auc': 0.7495632,
  'binaryaccuracy': 0.89375,
  'sensibility': 0.5882353,
  'specificity': 0.9108911}]

In [ ]:
# métricas de predicción para el conjunto de pruebas
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.76367074,
 'binaryaccuracy': 0.9025,
 'sensibility': 0.6111111,
 'specificity': 0.9162304}

**Interpretation**
The embedded space method here uses a mapping process of the bag information, where a mapping function is used to turn the bag into a vector summarizing the characteristics of the bag. In this case, a discriminative mapping `disc_mapping` from `MILESMapping()`, besides a Support Vector Classifier for the classification task. The trained model reached a high score of `BinaryAccuracy` of more than 90%. Besides, the area under the ROC curve is more than 0.76 which far higher than the performance of a random classifier (0.5). Both the `Binaryaccuracy` and `area under the ROC curve` indicate the overall considerable performance of the embedded classifier. Additionally, the specificity is superior of value 91% which demonstrate the high discrimination of the negative class, while the sensibility is 61% which relatively considerable, indicate the good performance of the model in identifying the positive class. We notice that the performance of the embedded space based method is good however its effectiveness in identifyin the positive class could be better.

### **CorelDessert Dataset**


In [ ]:

from sklearn.model_selection import train_test_split

bag_indices = range(len(bagsDesserts))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsDesserts, labelsDesserts, bag_indices, test_size=0.2, random_state=42
)

In [ ]:
# instantiate trainer
trainer = Trainer()

# preparing trainer
metrics = [AUC, BinaryAccuracy,Sensibility, Specificity]
model = SVC(kernel='linear', C=1, class_weight='balanced')
pipeline = [('scale', StandarizerBagsList()), ('disc_mapping', MILESMapping())]
trainer.prepare(model, preprocess_pipeline=pipeline ,metrics=metrics)


In [ ]:
# fitting trainer
valid = KFold(n_splits=5, shuffle = True)
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

5/5 [==============================] - 28s 6s/step - train_auc: 0.9918 - train_binaryaccuracy: 0.9844 - train_sensibility: 1.0000 - train_specificity: 0.9835 - val_auc: 0.8519 - val_binaryaccuracy: 0.9594 - val_sensibility: 0.7333 - val_specificity: 0.9705


In [ ]:
# printing validation results for each fold
print("The mean of the binary accuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binary accuracy =  0.96187496


[{'auc': 0.852833,
  'binaryaccuracy': 0.96875,
  'sensibility': 0.7222222,
  'specificity': 0.98344374},
 {'auc': 0.88803947,
  'binaryaccuracy': 0.95625,
  'sensibility': 0.8095238,
  'specificity': 0.9665552},
 {'auc': 0.8322369,
  'binaryaccuracy': 0.9625,
  'sensibility': 0.6875,
  'specificity': 0.9769737},
 {'auc': 0.78030306,
  'binaryaccuracy': 0.9625,
  'sensibility': 0.5833333,
  'specificity': 0.97727275},
 {'auc': 0.8519126,
  'binaryaccuracy': 0.959375,
  'sensibility': 0.73333335,
  'specificity': 0.9704918}]

In [ ]:
# predicting metrics for the test set
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.90488654,
 'binaryaccuracy': 0.97,
 'sensibility': 0.8333333,
 'specificity': 0.9764398}

**Interpretation:** The embedded space based method on the `CorelDesserts` dataset has an excellent global performance with a `Binaryaccuracy` of 97%, associated with a high value of the `area under the ROC curve` up to 0.90 which is considerable indicating the significative effectiveness in discriminating between the negative and the positive class. Besides, the model attained a `sensibility` of 83% demonstrating the percentage of positive bags correctly classified as positive, the value is notable indicating the significative performance in idetifyin the positive bags. Moreover, the model is highly identifying the negative class attaining a specificity value of 97%. We observe the significantly high performance of the embedded space based model in the `CorelDesserts` datasets, yet its classification performance in this dataset is better than in the previous dataset.

### **CorelCars Dataset**

In [ ]:
from sklearn.model_selection import train_test_split

bag_indices = range(len(bagsCars))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsCars, labelsCars, bag_indices, test_size=0.2, random_state=42
)

In [ ]:
# instantiate trainer
trainer = Trainer()

# preparing trainer
metrics = [AUC, BinaryAccuracy,Sensibility, Specificity]
model = SVC(kernel='linear', C=1, class_weight='balanced')
pipeline = [('scale', StandarizerBagsList()), ('disc_mapping', MILESMapping())]# obtaining a unique vector representing all the bag
trainer.prepare(model, preprocess_pipeline=pipeline ,metrics=metrics)


In [ ]:
# fitting trainer
valid = KFold(n_splits=5, shuffle = True)
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

5/5 [==============================] - 38s 7s/step - train_auc: 0.9744 - train_binaryaccuracy: 0.9516 - train_sensibility: 1.0000 - train_specificity: 0.9488 - val_auc: 0.8474 - val_binaryaccuracy: 0.9375 - val_sensibility: 0.7500 - val_specificity: 0.9448


In [ ]:
# printing validation results for each fold
print("The mean of the binary accuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binary accuracy =  0.9225


[{'auc': 0.8071039,
  'binaryaccuracy': 0.934375,
  'sensibility': 0.6666667,
  'specificity': 0.947541},
 {'auc': 0.774671,
  'binaryaccuracy': 0.909375,
  'sensibility': 0.625,
  'specificity': 0.9243421},
 {'auc': 0.8503735,
  'binaryaccuracy': 0.909375,
  'sensibility': 0.78571427,
  'specificity': 0.9150327},
 {'auc': 0.8294915,
  'binaryaccuracy': 0.921875,
  'sensibility': 0.72,
  'specificity': 0.938983},
 {'auc': 0.84740263,
  'binaryaccuracy': 0.9375,
  'sensibility': 0.75,
  'specificity': 0.9448052}]

In [ ]:
# predicting metrics for the test set
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.8747818,
 'binaryaccuracy': 0.9125,
 'sensibility': 0.8333333,
 'specificity': 0.9162304}

**Interpretation:**
The scores of the evaluation metrics (`Binaryaccuracy`, `auc`, `sensibility`, and `specificity`) of the built embedded space based model on the `CorelCars` are higher than 0.83, indicating the high performance on classifying the test bags. First, the `auc` and the `binaryaccuracy` are, repectively, 0.91 and 0.87 demonstracting the considerable overall performance of the classification model. Noticing that the `specificity` of the model is 0.91 indicating the significative classification ability of the positive bags. However, by seeing the `sensibility` of the model which is considerable  but less than `specificity` value, indicating the lower classification ability regarging  the bags of the positive class. Overall, the discriminative mapping using SVC classifier is quite good performing in the `CorelCars`.

## **3.3.  MILES:  Multiple-Instance Learning via Embedded Instance Selection**

In [ ]:
from mil.validators import LeaveOneOut
from mil.models import MILES

### **CorelAfrican Dataset**

In [ ]:

from sklearn.model_selection import train_test_split

#  'bags' is the feature matrix (bags x instances) and 'labels' is the label matrix (bags x labels)
bag_indices = range(len(bagsAfrican))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsAfrican, labelsAfrican, bag_indices, test_size=0.2, random_state=42
)


In [ ]:
trainer =Trainer()
valid =valid = KFold(n_splits=5, shuffle = True)
metrics = [AUC, BinaryAccuracy, Sensibility, Specificity]
model = MILES()
pipeline = [('scale', StandarizerBagsList())]

trainer.prepare(model, preprocess_pipeline=pipeline, metrics=metrics)

In [ ]:
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/5 [=====>........................] - ETA: 1:43 - train_auc: 0.9081 - train_binaryaccuracy: 0.8250 - train_sensibility: 1.0000 - train_specificity: 0.8162 - val_auc: 0.8303 - val_binaryaccuracy: 0.7656 - val_sensibility: 0.9048 - val_specificity: 0.7559

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/5 [===========>..................] - ETA: 1:15 - train_auc: 0.9094 - train_binaryaccuracy: 0.8281 - train_sensibility: 1.0000 - train_specificity: 0.8188 - val_auc: 0.8799 - val_binaryaccuracy: 0.8281 - val_sensibility: 0.9375 - val_specificity: 0.8224

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


3/5 [=================>............] - ETA: 51s - train_auc: 0.8983 - train_binaryaccuracy: 0.8078 - train_sensibility: 1.0000 - train_specificity: 0.7967 - val_auc: 0.8831 - val_binaryaccuracy: 0.7750 - val_sensibility: 1.0000 - val_specificity: 0.7662 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


4/5 [=======================>......] - ETA: 26s - train_auc: 0.9066 - train_binaryaccuracy: 0.8227 - train_sensibility: 1.0000 - train_specificity: 0.8132 - val_auc: 0.8636 - val_binaryaccuracy: 0.8469 - val_sensibility: 0.8824 - val_specificity: 0.8449

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


5/5 [==============================] - 126s 25s/step - train_auc: 0.9086 - train_binaryaccuracy: 0.8266 - train_sensibility: 1.0000 - train_specificity: 0.8171 - val_auc: 0.8553 - val_binaryaccuracy: 0.8375 - val_sensibility: 0.8750 - val_specificity: 0.8355


In [ ]:
print("The mean of the binaryaccuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binaryaccuracy =  0.81062496


[{'auc': 0.83030736,
  'binaryaccuracy': 0.765625,
  'sensibility': 0.9047619,
  'specificity': 0.7558528},
 {'auc': 0.8799342,
  'binaryaccuracy': 0.828125,
  'sensibility': 0.9375,
  'specificity': 0.82236844},
 {'auc': 0.8831169,
  'binaryaccuracy': 0.775,
  'sensibility': 1.0,
  'specificity': 0.76623374},
 {'auc': 0.86361873,
  'binaryaccuracy': 0.846875,
  'sensibility': 0.88235295,
  'specificity': 0.8448845},
 {'auc': 0.8552632,
  'binaryaccuracy': 0.8375,
  'sensibility': 0.875,
  'specificity': 0.8355263}]

In [ ]:
labels_predicted = trainer.predict(bags_test)
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.7789413,
 'binaryaccuracy': 0.78,
 'sensibility': 0.7777778,
 'specificity': 0.7801047}

**Interpretation:**
The `MILES` (Multi-instance Learning) model attained a good `Binaryaccuracy` value of 0.78 on the test subset of the `CorelCars` dataset, besides it has a good `area under  the ROC curve` of 0.77. Both the `binaryaccuracy` and `area under the ROC curve` confirm the good classification performance of the MILES model. Considering that `sensibility` and `specificity` have approximatively similar values of 0.77 and 0.78, respectively. The model has a good classification performance of the positive class bags as well as the negative class bags. Hence, a balanced classification ability on both negative and positive class bags. We notice the relatively considerable performance of the bags-space based model on the `CorelAfrican` dataset.

### **CorelDesserts Dataset**

In [ ]:
from sklearn.model_selection import train_test_split

#  'bags' is the feature matrix (bags x instances) and 'labels' is the label matrix (bags x labels)
bag_indices = range(len(bagsDesserts))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsDesserts, labelsDesserts, bag_indices, test_size=0.2, random_state=42
)

In [ ]:
trainer =Trainer()
valid =valid = KFold(n_splits=5, shuffle = True)
metrics = [AUC, BinaryAccuracy, Sensibility, Specificity]
model = MILES()
pipeline = [('scale', StandarizerBagsList())]

trainer.prepare(model, preprocess_pipeline=pipeline, metrics=metrics)

In [ ]:
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/5 [=====>........................] - ETA: 1:23 - train_auc: 0.9572 - train_binaryaccuracy: 0.9187 - train_sensibility: 1.0000 - train_specificity: 0.9143 - val_auc: 0.9128 - val_binaryaccuracy: 0.9469 - val_sensibility: 0.8750 - val_specificity: 0.9507

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/5 [===========>..................] - ETA: 1:01 - train_auc: 0.9608 - train_binaryaccuracy: 0.9258 - train_sensibility: 1.0000 - train_specificity: 0.9217 - val_auc: 0.8874 - val_binaryaccuracy: 0.9062 - val_sensibility: 0.8667 - val_specificity: 0.9082

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


3/5 [=================>............] - ETA: 41s - train_auc: 0.9675 - train_binaryaccuracy: 0.9383 - train_sensibility: 1.0000 - train_specificity: 0.9351 - val_auc: 0.8515 - val_binaryaccuracy: 0.9062 - val_sensibility: 0.7895 - val_specificity: 0.9136 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


4/5 [=======================>......] - ETA: 21s - train_auc: 0.9619 - train_binaryaccuracy: 0.9273 - train_sensibility: 1.0000 - train_specificity: 0.9237 - val_auc: 0.9089 - val_binaryaccuracy: 0.9125 - val_sensibility: 0.9048 - val_specificity: 0.9130

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


5/5 [==============================] - 104s 21s/step - train_auc: 0.9578 - train_binaryaccuracy: 0.9203 - train_sensibility: 1.0000 - train_specificity: 0.9156 - val_auc: 0.8670 - val_binaryaccuracy: 0.9125 - val_sensibility: 0.8182 - val_specificity: 0.9159


In [ ]:
print("The mean of the binaryaccuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binaryaccuracy =  0.916875


[{'auc': 0.912829,
  'binaryaccuracy': 0.946875,
  'sensibility': 0.875,
  'specificity': 0.9506579},
 {'auc': 0.8874317,
  'binaryaccuracy': 0.90625,
  'sensibility': 0.8666667,
  'specificity': 0.90819675},
 {'auc': 0.85154754,
  'binaryaccuracy': 0.90625,
  'sensibility': 0.7894737,
  'specificity': 0.91362125},
 {'auc': 0.9089027,
  'binaryaccuracy': 0.9125,
  'sensibility': 0.9047619,
  'specificity': 0.9130435},
 {'auc': 0.86701965,
  'binaryaccuracy': 0.9125,
  'sensibility': 0.8181818,
  'specificity': 0.9158576}]

In [ ]:
labels_predicted = trainer.predict(bags_test)
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.9064863,
 'binaryaccuracy': 0.9225,
 'sensibility': 0.8888889,
 'specificity': 0.92408377}

**Interpretation:** The evaluation of the MILES model predictions on the test data of the `CorelDesserts` indicate at the first sight superior performance. It attained an excellent score of `binaryaccuracy` 92% and a high value of the `area under the ROC curve` of 0.90. Those both metrics assesses the global classification effectiveness of the `MILES` model and indicating that is overall significantly performing on test data. Besides, 92% of the negative bags are correctly classified as negative which is quite significant. The ratio of correctly classified positive bags as postive is 88% indicating the considerable identification of the negative bags. Thus, the ratio of identification of the negative bags is a little bit greater than that of the positive bags. The performance of the MILES model on the `CorelDesserts` dataset is higher than that on the `CorelAfrican`.

### **CorelCars Dataset**

In [ ]:
from sklearn.model_selection import train_test_split

#  'bags' is the feature matrix (bags x instances) and 'labels' is the label matrix (bags x labels)
bag_indices = range(len(bagsCars))
bags_train, bags_test, labels_train, labels_test, indices_train, indices_test = train_test_split(
    bagsCars, labelsCars, bag_indices, test_size=0.2, random_state=42
)

In [ ]:
trainer =Trainer()
valid =valid = KFold(n_splits=5, shuffle = True)
metrics = [AUC, BinaryAccuracy, Sensibility, Specificity]
model = MILES()
pipeline = [('scale', StandarizerBagsList())]

trainer.prepare(model, preprocess_pipeline=pipeline, metrics=metrics)

In [ ]:
history = trainer.fit(bags_train, labels_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/5 [=====>........................] - ETA: 1:59 - train_auc: 0.9228 - train_binaryaccuracy: 0.8531 - train_sensibility: 1.0000 - train_specificity: 0.8456 - val_auc: 0.8483 - val_binaryaccuracy: 0.8469 - val_sensibility: 0.8500 - val_specificity: 0.8467

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/5 [===========>..................] - ETA: 1:10 - train_auc: 0.9150 - train_binaryaccuracy: 0.8398 - train_sensibility: 1.0000 - train_specificity: 0.8300 - val_auc: 0.8173 - val_binaryaccuracy: 0.7625 - val_sensibility: 0.8750 - val_specificity: 0.7596

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


3/5 [=================>............] - ETA: 52s - train_auc: 0.9037 - train_binaryaccuracy: 0.8164 - train_sensibility: 1.0000 - train_specificity: 0.8074 - val_auc: 0.8967 - val_binaryaccuracy: 0.8469 - val_sensibility: 0.9545 - val_specificity: 0.8389 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


4/5 [=======================>......] - ETA: 26s - train_auc: 0.9152 - train_binaryaccuracy: 0.8391 - train_sensibility: 1.0000 - train_specificity: 0.8303 - val_auc: 0.8240 - val_binaryaccuracy: 0.8344 - val_sensibility: 0.8125 - val_specificity: 0.8355

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


5/5 [==============================] - 136s 27s/step - train_auc: 0.9263 - train_binaryaccuracy: 0.8602 - train_sensibility: 1.0000 - train_specificity: 0.8526 - val_auc: 0.8191 - val_binaryaccuracy: 0.8250 - val_sensibility: 0.8125 - val_specificity: 0.8257


In [ ]:
print("The mean of the binaryaccuracy = ",np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
history['metrics_val']

The mean of the binaryaccuracy =  0.823125


[{'auc': 0.84833336,
  'binaryaccuracy': 0.846875,
  'sensibility': 0.85,
  'specificity': 0.8466667},
 {'auc': 0.8173078,
  'binaryaccuracy': 0.7625,
  'sensibility': 0.875,
  'specificity': 0.75961536},
 {'auc': 0.89673585,
  'binaryaccuracy': 0.846875,
  'sensibility': 0.95454544,
  'specificity': 0.8389262},
 {'auc': 0.82401323,
  'binaryaccuracy': 0.834375,
  'sensibility': 0.8125,
  'specificity': 0.8355263},
 {'auc': 0.8190789,
  'binaryaccuracy': 0.825,
  'sensibility': 0.8125,
  'specificity': 0.8256579}]

In [ ]:
labels_predicted = trainer.predict(bags_test)
trainer.predict_metrics(bags_test, labels_test)

{'auc': 0.85703903,
 'binaryaccuracy': 0.7775,
 'sensibility': 0.9444444,
 'specificity': 0.76963353}

**Interpretation:** The `MILES` performance measures on the test data are stated in the output of the cell above. We notice that the `binaryaccuracy` value is good (0.77), however lower than that on the other datasets `CorelAfrican` and `CorelDesserts`. It indicates that the `MILES` model correctly label  77% of the bags considering the class distribution. Moreover, the `area under the ROC curve` is 0.85, indicating the significant capacity of the `MILES` model in discriminating between the negative and the positive class bags. Besides, the sensibility is excellent, where the model correctly identified 94% of the positive bags. While, it only correctly labels 76% of the negative bags, which is relatively lower than the that of the positive bags. Hence, the model is well performing with respect to the `CorelCars` dataset however is a bit biased to the positive class bags, which lowers the `binaryaccuracy`.

## **4. Comparative Analysis and Conclusion**


**Description** In this notebook of the Multi-Instance Classification Task. We discovered the resources available related to the task including the repositories of the multi-instance datasets and the two existing multi-instance learning python libraries (`mil` and `misvm`). Consequently, we chose three datasets `CorelAfrican`, `CorelDesserts`, and `CorelCars` along with three multi-instance algorithms. The first model is an instance based method, the rest are embedded-space based algorithm: one called the Discriminative Mapping using Support Vector Classifier and the second model is the MILES.




 **Comparative Analysis**

 First of all the APR method which based on the standard assumption of the positive bag is at least contains on positive instance is severely biased towards the positive class. As such it attained a perfect performance in identifying the positive bags while almost ignoored the majority of the negative bags. Hnece the APR regarding the employed multi-instance datasets is the least classification performance by misclassifying the negative bags.

On the other hand,The discriminative mapping using SVC and the MILES are similarly and significantly performing on the `CorelDessers` dataset in term of `area under the ROC curve`, however the MILES model is less biased to the positive bags class compared to the other model. On the datasets `CorelAfrican` and `CorelCars`, the discriminative mapping using SVC model is exceeding the MILE in term `specificity` and `binaryaccuracy`, indicating that it is achieving higher general performance and more significant identification of the negative bags class. Nevertheless, the MILES is releasing better recognition of positive bags in both datasets (`CorelAfrican` and `CorelCars`). Both models are releasing close values of the `area under the ROC curve` in both datasets indicating the insignificant difference in their discrimination between negative and positive classes of bags. Consequently, the MILES is better identifying the positive bags class in all used datasets associated with relatively considerable to excellent ratios of the negative class classification. The discriminative mapping using SVC is more biased to the negative class of bags which comes to the cost of the classification of positive class bags. The MILES reaches a better tradeoff between the correct classification of the classes of bags regarding the used datasets.

Finally, the embedded space methods: discriminative mapping using SVC and the MILES released significantly better performance compared to the APR model, but the MILES is providing better compromise between the specificity and sensibility.


**Conclusion:** The experimental procedures with multi-instance classification included two different approaches, instance-space based and embedded-space based, respectively, APR and (MILES and the discriminative mapping using SVC). The trainer process including a pipeline containing the used methods Standardization of the bag list as preprocessing as well as the cross validation process for evaluation with the selected metrics.The APR significantly misclassified the negative bags and provided unreliable predictions, hence it is considered the least effective multi-instance classifier with respect to the employed datasets. The results of the embedded-space methods on the selected data is generally good at first glance. The comparison released their similar performance in one dataset. On the other datasets, the MILES attained a better compromise between the identification of negative bags as well as the positive bags, further the discriminative mapping using SVC is noticable biased towards the majority class that is the negative class of bags. All in all, the embedded-space methods achieved high performance in the three multi-instance classification datasets, while the instance-based method stands as the weak classifier.

 ***Nota: dados los problemas actuales con la librería python misvm, no hemos podido utilizar ningún método del enfoque misvm.***

## **References**


 **Datasets:**
* [Datasets_figshare](https://figshare.com/articles/dataset/MIProblems_A_repository_of_multiple_instance_learning_datasets/6633983)
* [multipleinstance](http://www.multipleinstancelearning.com/datasets/)

**Python Library:**

* Multiple instance learning library:
[mil_github](https://github.com/rosasalberto/mil)

 **Articles:**
* [MILES: Multiple-Instance Learning
via Embedded Instance Selection](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=1717454)
* [Multiple instance classification: Review, taxonomy and comparative study](https://www.sciencedirect.com/science/article/pii/S0004370213000581)